### Package installations

In [ ]:
! pip3 install --upgrade pip

! pip3 install pandas -q
! pip3 show pandas | grep -Ei 'Name:|Version:'

## 1 Clean dataset for special characters, and output csv

In [ ]:
import pandas as pd
import re

# Clean the 'abstract' column for weird substrings
def clean_abstract(text):
    if pd.isnull(text):
        return text

    text = text.replace('"', '').replace('\\', '')

    # Pattern to match substrings starting with '(', containing chars, followed by 'http', and ending with ')', or that start and ending with commas, that also containins 'http'
    link_pattern_xor = r'\([^)]*http[^)]*\)|,([^,]*http[^,]*),'
    isolated_link_pattern = r'https?://\S+(?=\s|$)'
    # Regex pattern for replacing trailing spaces with a single space
    trailing_space_pattern = r' {2,}'

    # Removes links within parentheses and commas, then replaces trailing spaces with single space
    text = re.sub(trailing_space_pattern, ' ', re.sub(isolated_link_pattern, '', re.sub(link_pattern_xor, '', text)))

    return text



# Replaces special characters on text columns
def clean_special_substrings(text):
    # Removes special characters and weird substrings
    text = text.replace(' v/ ', ' ').replace('\\n', ' ').replace('▬', '').replace('\u00A0', ' ')

    return text


def clean_hash_delimiter(text):
    text = text.replace('###', ',')

    return text


def clean_constraints_column(text):
    if text == '###':
        text = text.replace('###', '')
    else:
        text = text.replace('###', ',')
    
    return text


def clean_security_constraints(text):
    text = text.replace('#########', '').replace('######', ',').replace('###', ',')

    return text


def clean_legal_constraints(text):
    text = text.replace('######', ',').replace('###', ',')

    return text


def clean_contact(text):
    text = text.replace('###', ',')

    return text


def clean_links(text):
    if text[:3] == '###':
        text = text[3:]

    text = text.replace('###', ',')
    
    return text


def clean_product_info(text):
    text = text.replace('###', ',')

    return text



dataset_file = 'Metadata_excel.xlsx'
df = pd.read_excel(dataset_file)

# Fill NaN values with an empty string
df.fillna('', inplace=True)

columns_clean_special_chars = [
    'schema', 'uuid', 'hierarchyLevel', 'title', 'abstract', 'keyword', 'geoBox', 'Constraints', 
    'SecurityConstraints', 'LegalConstraints', 'temporalExtent', 'responsibleParty',  
    'productInformation', 'parentId'
]
# Apply cleaning functions columns
for col in df.columns:
    # Apply cleaning to all string columns
    if df[col].dtype == 'object':
        if col in columns_clean_special_chars:
            df[col] = df[col].apply(clean_special_substrings)
            #df[col] = df[col].apply(replace_norwegian_characters)

        # Include cleaning of weird substrings
        if col == 'abstract':
            df[col] = df[col].apply(clean_abstract)
        
        # Clean keyword column
        if col == 'keyword':
            df[col] = df[col].apply(clean_hash_delimiter)

        # Clean geoBox column
        if col == 'geoBox':
            df[col] = df[col].apply(clean_hash_delimiter)

         # Clean Constraints column
        if col == 'Constraints':
            df[col] = df[col].apply(clean_constraints_column)
       
        # Clean SecurityConstraints column
        if col == 'SecurityConstraints':
            df[col] = df[col].apply(clean_security_constraints)
 
        # Clean LegalConstraints column
        if col == 'LegalConstraints':
            df[col] = df[col].apply(clean_legal_constraints)
        
        # Clean LegalConstraints column
        if col == 'responsibleParty':
            df[col] = df[col].apply(clean_contact)

        # Clean productInformation column
        if col == 'productInformation':
            df[col] = df[col].apply(clean_product_info)

        # Clean links columns
        if col in ['image', 'link']:
            df[col] = df[col].apply(clean_links)

 
cleaned_csv_file = 'cleaned_metadata.csv'
df.to_csv(cleaned_csv_file, sep='|', index=False)
df.head()